In [1]:
# BTD save decrypter
# from https://github.com/codeshaunted/monke

import zlib
import hashlib
from Crypto.Cipher import AES
import os

DUMMY_HEADER_LENGTH = 44
PASSWORD_INDEX_LENGTH = 8
SALT_LENGTH = 24
KEY_LENGTH = 16
IV_LENGTH = 16
DERIVE_ITERATIONS = 10
PASSWORD = b'11'
algorithm = 'AES'
mode = 'CBC'

def pbkdf2_hmac_sha1(password, salt, iterations, key_length):
    return hashlib.pbkdf2_hmac('sha1', password, salt, iterations, key_length)

def unpack(bytes_data):
    salt = bytes_data[DUMMY_HEADER_LENGTH + PASSWORD_INDEX_LENGTH:DUMMY_HEADER_LENGTH + PASSWORD_INDEX_LENGTH + SALT_LENGTH]
    encrypted_bytes = bytes_data[DUMMY_HEADER_LENGTH + PASSWORD_INDEX_LENGTH + SALT_LENGTH:]

    derived_bytes = pbkdf2_hmac_sha1(PASSWORD, salt, DERIVE_ITERATIONS, KEY_LENGTH + IV_LENGTH)

    iv = derived_bytes[:16]
    key = derived_bytes[16:32]

    cipher = AES.new(key, AES.MODE_CBC, iv)
    decrypted = cipher.decrypt(encrypted_bytes)

    decompressed = zlib.decompress(decrypted)

    return decompressed.decode('utf-8')

def pack(json_data):
    dummy_header = bytes([0]) * DUMMY_HEADER_LENGTH

    password_index = b'\x02' + bytes([0]) * (PASSWORD_INDEX_LENGTH - 1)

    salt = os.urandom(SALT_LENGTH)

    derived_bytes = pbkdf2_hmac_sha1(PASSWORD, salt, DERIVE_ITERATIONS, KEY_LENGTH + IV_LENGTH)

    iv = derived_bytes[:16]
    key = derived_bytes[16:32]

    compressed = zlib.compress(json_data.encode('utf-8'), level=3)

    cipher = AES.new(key, AES.MODE_CBC, iv)
    encrypted = cipher.encrypt(compressed)

    return dummy_header + password_index + salt + encrypted

In [3]:
from shortcuts.file_io import choose_file
profilesave = choose_file()
# ~/Library/Application Support/Steam/userdata/(STEAM ID))/960090/local/link/PRODUCTION/current
with open(profilesave, 'rb') as f:
    binary_file = f.read()

save = unpack(binary_file)

import pprint
pprint.pprint(save[1:])

('{"version":14,"savedByGameVersion":"42.2","savedBySkuId":35,"towerXp":{"DartMonkey":2934255.9336799532,"BombShooter":6064552.0235916078,"TackShooter":2040746.092304945,"IceMonkey":1727453.5698150396,"BoomerangMonkey":4942992.42605859,"GlueGunner":224847.49750202894,"HeliPilot":11825775.280617476,"SniperMonkey":1805256.0883641243,"WizardMonkey":4398401.6422031224,"NinjaMonkey":11357668.411994159,"Alchemist":9910177.2282034457,"Druid":3219325.3769405782,"MonkeyAce":4175125.1819009781,"BananaFarm":12254326.076169729,"BananaFarmer":0.0,"MonkeySub":12582848.439928621,"MonkeyVillage":5926626.182887435,"SpikeFactory":5687144.3352867961,"SuperMonkey":7879443.6143624783,"Natures '
 'Ward Totem":0.0,"MonkeyBuccaneer":3396469.9521989822,"Sun '
 'Avatar":0.0,"BuccaneerPlaneV1":0.0,"NaturesWardTotem":0.0,"MortarMonkey":3665401.4416162372,"Marine":0.0,"BuccaneerPlaneV2":0.0,"SacrificialTotem":0.0,"Wizard-LordPhoenix":0.0,"PortableLake":0.0,"TechBot":0.0,"EngineerMonkey":205501.34960365295,"Sentry"

In [5]:
psave = pack(save[1:])
print(psave)

ValueError: Data must be padded to 16 byte boundary in CBC mode